<a href="https://colab.research.google.com/github/ak283/RAG-based-Q-A-application-using-LangChain/blob/main/RAG_based_Q%26A_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain_community sentence_transformers
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [2]:
!pip install langchain-google-genai
!pip install tiktoken
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 3.9 MB/s eta 0:00:00


In [3]:
!pip install langchain_pinecone

In [55]:
from google.colab import userdata
GOOGLE_API_KEY= userdata.get('GOOGLE_API_KEY')
PINECONE_API_KEY= userdata.get('PINECONE_API_KEY')

In [57]:
import google.generativeai as genai

In [58]:
#genai.configure(api_key=GOOGLE_API_KEY)

In [59]:
import os
os.environ['GOOGLE_API_KEY']=GOOGLE_API_KEY

In [60]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('Ashish_Kumar.pdf')
docs=loader.load()

In [61]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

doc = split_docs(docs)
print(len(doc))

7


In [62]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

In [63]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key= PINECONE_API_KEY)

pc.delete_index("my-rag-project")

pc.create_index(
    name="my-rag-project",
    dimension=768, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


In [64]:
from langchain_pinecone import PineconeVectorStore

os.environ['PINECONE_API_KEY'] = '22e7ba43-9f7e-4f62-9727-843dc24ea71c'

index_name = "my-rag-project"


abc= PineconeVectorStore.from_documents(
        documents=doc,
        index_name=index_name,
        embedding=embeddings
    )

In [65]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key =GOOGLE_API_KEY)


In [66]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=abc.as_retriever()
)


In [67]:
query="summerize ashish resume?"

In [68]:
qa.run(query)

"Please provide me with Ashish's resume so I can summarize it for you. \n"

In [69]:
import textwrap
from IPython.display import Markdown

In [70]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [76]:
query="summerize ashish resume?"
to_markdown(qa.run(query))

> Ashish is an AI/ML enthusiast with a strong foundation in Python and SQL. He has experience in automation testing, data analysis, and machine learning. He has worked on projects involving web scraping, sentiment analysis using BERT, and building a user-friendly platform on Anaplan. He is proficient in various machine learning algorithms and familiar with GenAI concepts and LLM models. 


In [75]:
query="What are the AI/ML projects done by ashish?"
to_markdown(qa.run(query))

> Ashish has done two AI/ML projects:
> 
> 1. **Sentiment analysis of Amazon food reviews using BERT model:** This project involved analyzing customer text reviews and comments on Amazon to determine sentiment. It utilized Python, NumPy, Pandas, Hugging Face, PyTorch, Transformers, Tokenizer, and BERT.
> 
> 2. **Analysis of various companies listed on AmbitionBox website through web scraping:** This project scraped data from the AmbitionBox website, including sector, reviews, ratings, employee counts, and location. It used Python, Pandas, Requests, Beautiful Soup, web scraping, and data analysis. The code for this project is available on GitHub: https://github.com/ak283/-Ambitionbo x.com-web-scr aping.git 


In [73]:
query="How many year of experience ashish have?"
to_markdown(qa.run(query))

> Ashish has a total of **2 years** of experience. 
> 
> Here's how we know:
> 
> * **Cognizant Technology Solutions:**  Program Analyst from March 2021 to January 2023 (approximately 2 years)
> * **Feynn Labs Services:** Machine Learning Intern from September 2022 to January 2023 (approximately 4 months) 
> 
> While the internship is shorter, it still counts towards his overall experience. 


In [74]:
query="What are the skills of ashish?"
to_markdown(qa.run(query))

> Ashish's skills include:
> 
> **Programming Languages:**
> * Python
> * SQL
> 
> **Data Analysis and Manipulation:**
> * Exploratory data analysis
> * Data cleaning and preprocessing
> 
> **Machine Learning Algorithms:**
> * Linear regression
> * Logistic regression
> * K-nearest neighbors
> * Random forest
> * Decision trees
> * K-means clustering
> 
> **Generative AI Concepts and LLM Models:**
> * Familiar with a range of GenAI concepts and LLM models
> 
> **Tools:**
> * Microsoft Azure
> * Snowflake
> * LangChain
> 
> **Packages:**
> * NumPy
> * Pandas
> * Matplotlib
> * Seaborn
> * PySpark
> * Scikit-learn
> * Spacy
> * PyTorch
> * TensorFlow
> * Keras
> * NLTK
> * Hugging Face
> 
> **Other Skills:**
> * Web scraping
> * Sentiment analysis
> * Automation testing
> * Data validation
> * Defect identification and logging
> * API usage
> * Microsoft Excel
> * AWS
> * Anaplan
> 
> **Project Experience:**
> * Web scraping project using Python, Pandas, Requests, Beautiful Soup
> * Sentiment analysis project using Python, NumPy, Pandas, Hugging Face, PyTorch, Transformer, Tokenizer, BERT
> * Lulemon project using SQL, Python, AWS, Anaplan
> * Network Rail project using SQL, Python, Postman, Ms Excel, API, Microsoft Azure
